In [ ]:
import pandas as pd
import numpy as np
#nice merge on municipality name
train_data = pd.read_csv('../data/stores_train.csv')

grunnkrets_stripped = pd.read_csv('../data/grunnkrets_norway_stripped.csv')

train_data = pd.merge(train_data, grunnkrets_stripped[['grunnkrets_id', 'municipality_name']], on='grunnkrets_id', how='left')

train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')
train_data = train_data.reset_index()

#create id as int for plaace hierarchy_id
train_data['id1'] = train_data['plaace_hierarchy_id'].apply(lambda x: int(x[0]))
train_data['id2'] = train_data['plaace_hierarchy_id'].apply(lambda x: int(x[2]))
train_data['id3'] = train_data['plaace_hierarchy_id'].apply(lambda x: int(x[4]))

#drop everything that is not needed
train_data = train_data.drop(['year', 'store_name', 'plaace_hierarchy_id', 'sales_channel_name', 'address', 'chain_name', 'mall_name'], axis=1)
df = train_data

# Dist function
def dist(lat, lon, df): #distance from (lat, lon) to all other stores in the dataframe
    lat1 = abs(df['lat'] - lat)
    lon1 = abs(df['lon'] - lon)
    eucdist = np.sqrt(lon1**2 + lat1**2)
    print(type(eucdist))
    return eucdist
    



In [ ]:



# Mean revenue for municipality and level
def mean_muni_lvl():
    municipalities = train_data[["municipality_name", "revenue"]].groupby(
        ["municipality_name"]
    ).mean().reset_index()
    municipalities = municipalities.rename(columns={'revenue':'mean_revenue_for_municipality_and_level'})
    train_data = train_data.merge(municipalities, how="left", on=["municipality_name"])
    # we get a bunch of duplicates of store_ids...? Remove them.
    train_data = train_data.drop_duplicates(subset=['store_id'], keep='first')
    train_data.head()

# Number of stores within a distance and same lvl

distance = 10
df['num_stores_within_distance_and_same_lvl'] = 0
df['num_stores_within_distance'] = 0
num = 0 

for index in range(len(df)):
    lat = df._get_value(index,'lat')
    lon = df._get_value(index,'lon')
    lvl = df._get_value(index, 'id1')
    dist_to_all = dist(lat, lon, df)

    count_distance_and_lvl = 0 #for distance and level
    count_distance = 0 #only for distance
    iter = 0
    
    for number in dist_to_all:
        if number < distance:
            count_distance += 1
            if lvl == df._get_value(iter, 'id1'):
                count_distance_and_lvl += 1
        iter += 1

    df._set_value(index, 'num_stores_within_distance_and_same_lvl', count_distance_and_lvl)
    df._set_value(index, 'num_stores_within_distance', count_distance)

    num+=1 #for runtime tracking only
    print(str(num) + "/12860")
    print(count_distance == count_distance_and_lvl)




#number_within_distance(3)
#number_within_distance_and_lvl(3, "id1")
df.head(50)



In [37]:
# mean revenue same lvl and distances to busstops
from dis import dis
import math
import geopandas as gpd
import numpy
from shapely import wkt
from shapely import wkb

busstops = pd.read_csv('../data/busstops_norway.csv')

df['dist_closest_buss < 200m'] = 0
df['dist_closest_buss < 500m'] = 0
df['dist_closest_buss < 1000m'] = 0
df['dist_closest_buss > 1000m'] = 0


# for index in range(len(df)):
#     lat = df._get_value(index,'lat')
#     lon = df._get_value(index,'lon')
#     lvl = df._get_value(index, 'id1')
#     val = busstops._get_value(index,'geometry')
#         dist = geopy.distance.geodesic((val.x, val.y), (lat, lon)).km
#     buss_dist = dist()
busstops['geometry'] = gpd.GeoSeries.from_wkt(busstops['geometry'])
busstops['x'] = busstops.geometry.apply(lambda x: x.x)
busstops['y'] = busstops.geometry.apply(lambda x: x.y)


In [38]:
import numpy as np
import pandas as pd

df1 = pd.DataFrame({'node': ['abc', 'def'], 
                    'x': [645, 187], 
                    'y': [714, 754], 
                    'value': [8, 11]}) 
df2 = pd.DataFrame({'location': ['ijk', 'lmn'], 
                    'x': [621,  202], 
                    'y': [744, 720], 
                    'value': [1, -5]}) 

# compute Euclidean distances (using Pythagoras' theorem)
df2['distances'] = [[((df1['x'][j] - df2['x'][i]) ** 2 
                    + (df1['y'][j] - df2['y'][i]) ** 2) ** 0.5 
                     for j in range(len(df1))] 
                    for i in range(len(df2))]
df2

,location,x,y,value,distances
0,ijk,621,744,1,"[38.41874542459709, 434.11519208615584]"
1,lmn,202,720,-5,"[443.0406301909566, 37.16180835212409]"
